In [1]:
import numpy as np
import requests
import time

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [2]:
# Load the dataset and labels
x=np.load('MCx.npy')
y=np.load('MCy.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
et=0
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        model = RandomForest.trainClassifier(TrD, numClasses=np.unique(y).shape[0],categoricalFeaturesInfo={},numTrees=100) #The number of classes in the dataset
        predictions = model.predict(TsD.map(lambda x: x.features))
        st = time.time()
        labelsAndPredictions = TsD.map(lambda lp: lp.label).zip(predictions)
        lpa=np.vstack((lpa,labelsAndPredictions.toDF().toPandas()))
        et+=time.time()-st
        acc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(TsD.count())
        pp=pp+acc
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time requires to predict a label per each object in uS.

In [8]:
smr

[[43.0, 0.9176, 24.8514],
 [42.0, 0.9176, 24.3651],
 [41.0, 0.9176, 24.9002],
 [40.0, 0.9176, 34.0807],
 [39.0, 0.9176, 23.8789],
 [38.0, 0.9176, 25.0913],
 [37.0, 0.9176, 23.9276],
 [36.0, 0.9176, 24.0734],
 [35.0, 0.9175, 28.7908],
 [34.0, 0.9175, 23.9276],
 [33.0, 0.9175, 23.9275],
 [32.0, 0.9176, 24.2193],
 [31.0, 0.9177, 23.4898],
 [30.0, 0.9177, 23.9761],
 [29.0, 0.9177, 23.6357],
 [28.0, 0.9175, 25.3378],
 [27.0, 0.9175, 24.8516],
 [26.0, 0.9175, 24.3167],
 [25.0, 0.9175, 25.7756],
 [24.0, 0.9175, 24.9002],
 [23.0, 0.9175, 24.5598],
 [22.0, 0.9175, 24.8376],
 [21.0, 0.9175, 26.116],
 [20.0, 0.9175, 25.3504],
 [19.0, 0.9175, 25.2407],
 [18.0, 0.9175, 24.9975],
 [17.0, 0.9175, 24.9488],
 [16.0, 0.9174, 24.8172],
 [15.0, 0.9174, 24.7542],
 [14.0, 0.9174, 24.2679],
 [13.0, 0.9169, 24.1707],
 [12.0, 0.9152, 23.7329],
 [11.0, 0.8484, 21.4473],
 [10.0, 0.8484, 21.0581],
 [9.0, 0.8484, 20.8636],
 [8.0, 0.8455, 24.9488],
 [7.0, 0.8408, 21.0095],
 [6.0, 0.8361, 19.4533],
 [5.0, 0.829, 19.